# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [4]:
diff_schools = school_data_complete["school_name"].value_counts()
total_schools = diff_schools.count()

diff_id = school_data_complete["Student ID"].value_counts()
total_students = diff_id.count()

individual_budgets = school_data_complete["budget"].unique()
total_budget = individual_budgets.sum()

sum_math_scores = school_data_complete["math_score"].sum()
sum_math_scores
avg_math_score = sum_math_scores / total_students

sum_read_scores = school_data_complete["reading_score"].sum()
sum_read_scores
avg_reading_score = sum_read_scores / total_students

all_math_scores = school_data_complete["math_score"].count()
all_math_scores
good_math = school_data_complete[school_data_complete.math_score >= 70]
passing_math = good_math["math_score"].count()
passing_math
pass_math_percent = 100*passing_math / all_math_scores

all_read_scores = school_data_complete["reading_score"].count()
all_read_scores
good_read = school_data_complete[school_data_complete.reading_score >= 70]
passing_read = good_read["reading_score"].count()
passing_read
pass_read_percent = 100*passing_read / all_read_scores
overall_pass_rate = (avg_math_score + avg_reading_score) / 2

district_summary_df = pd.DataFrame({"Total Schools": [total_schools],
    "Total Students":[total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_reading_score],
    "% Passing Math": [pass_math_percent],
    "% Passing Reading": [pass_read_percent],
    "% Overall Passing Rate": [overall_pass_rate]})

district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
# Saved cell for groupby things
grouped_schools = school_data_complete.groupby(['school_name'])

avg_math_by_school = grouped_schools['math_score'].mean()
math_df = pd.DataFrame({"Average Math Score": avg_math_by_school})

avg_reading_by_school = grouped_schools['reading_score'].mean()
read_df = pd.DataFrame({"Average Reading Score": avg_reading_by_school})

math_and_read_df = pd.merge(math_df, read_df, on="school_name")

In [6]:
# Initial dataframe setup
all_names_and_types = school_data_complete[['school_name', 'type', 'budget']]
names_and_types = all_names_and_types.drop_duplicates('school_name')
df_1 = names_and_types.set_index('school_name')
# Total Students column
school_students_count = grouped_schools["Student ID"].count()
df_students = pd.DataFrame({"Total Students": school_students_count})
# Merge and Calculate *Per Student Budget*
df_merge_1 = pd.merge(df_1, df_students, on='school_name')
df_merge_1['Per Student Budget'] = df_merge_1['budget']/df_merge_1['Total Students']
# Merge 2: Electric Boogaloo
df_merge_2 = pd.merge(df_merge_1, math_and_read_df, on='school_name')
# Mathterpiece
pass_math = school_data_complete.loc[(school_data_complete["math_score"] > 69)]
pass_math_2 = pass_math.drop(['Student ID', 'student_name', 'gender', 'grade', 'reading_score', 'School ID', 'type', 'size', 'budget'], axis=1)
pass_math_students = pass_math_2.groupby(['school_name'])
pass_math_count = pass_math_students.count()
# I shall call him, "Mini Merge"
df_math_pass = pd.merge(pass_math_count, df_students, on='school_name')
df_math_pass["% Passing Math"] = 100*df_math_pass["math_score"] / df_math_pass["Total Students"]
df_math_percent = df_math_pass["% Passing Math"]
# Merge 3: Merge with a Vengeance
df_merge_3 = pd.merge(df_merge_2, df_math_percent, on='school_name')
# Take a look, It's in a book
pass_read = school_data_complete.loc[(school_data_complete["reading_score"] > 69)]
pass_read_2 = pass_read.drop(['Student ID', 'student_name', 'gender', 'grade', 'math_score', 'School ID', 'type', 'size', 'budget'], axis=1)
pass_read_students = pass_read_2.groupby(['school_name'])
pass_read_count = pass_read_students.count()
df_read_pass = pd.merge(pass_read_count, df_students, on='school_name')
df_read_pass["% Passing Reading"] = 100*df_read_pass["reading_score"] / df_read_pass["Total Students"]
df_read_percent = df_read_pass["% Passing Reading"]
# Merge 4: Live Free or [Merge]
df_merge_4 = pd.merge(df_merge_3, df_read_percent, on='school_name')
# We The Best
df_overall_pass = pd.merge(df_math_percent, df_read_percent, on='school_name')
df_overall_pass["% Overall Passing Rate"] = (df_overall_pass["% Passing Math"] + df_overall_pass["% Passing Reading"]) / 2
df_overall_percent = df_overall_pass["% Overall Passing Rate"]
# Merge 5: A Good Day to [Merge]
df_merge_5 = pd.merge(df_merge_4, df_overall_percent, on='school_name')

# Re-order columns for final dataframe summary
df_merge_5 = df_merge_5[['type', 'Total Students', 'budget', 'Per Student Budget',
       'Average Math Score', 'Average Reading Score', '% Passing Math',
       '% Passing Reading', '% Overall Passing Rate']]

# Prepare for the end
school_summary_df = df_merge_5.rename(columns={"type":"School Type", "budget":"Total School Budget"})
# Proper mapping
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:.2f}".format)
#school_summary_df = pd.set_option("display.colheader_justify", "left")

top_schools = school_summary_df.sort_values("% Overall Passing Rate", ascending=False)
top_5_schools = top_schools.head(5)
top_5_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [7]:
bottom_schools = school_summary_df.sort_values("% Overall Passing Rate")
bottom_5_schools = bottom_schools.head(5)
bottom_5_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [249]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [292]:
grade_breakdown = school_data_complete.groupby(['school_name', 'grade'])
math_populations = grade_breakdown["math_score"].count()
math_populations.head()
grades_df = pd.DataFrame({"Total Students": math_populations})
grades_df.head()

Total Students
school_name         grade                
Bailey High School  10th   1239          
                    11th   1251          
                    12th   1028          
                    9th    1458          
Cabrera High School 10th    466

In [294]:
pass_math_3 = pass_math.drop(['Student ID', 'student_name', 'gender', 'reading_score', 'School ID', 'type', 'size', 'budget'], axis=1)
pass_math_3.head()
pass_math_grouped = pass_math_3.groupby(['school_name', 'grade'])
pass_math_grouped.count().head()

math_score
school_name         grade            
Bailey High School  10th   822       
                    11th   856       
                    12th   661       
                    9th    979       
Cabrera High School 10th   438

## Reading Score by Grade 

* Perform the same operations as above for reading scores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [303]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 600, 620, 640, 660]
group_names = ["<$600", "$600-620", "$620-640", "$640-660"]

In [302]:
school_summary_df_2 = school_summary_df
school_summary_df_2["Budget Summary"] = pd.cut(school_summary_df_2["Per Student Budget"], spending_bins, labels=group_names)
school_summary_df_2

TypeError: '<' not supported between instances of 'int' and 'str'

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [8]:
school_summary_df.head()
slim_summary = school_summary_df.drop(['Total Students', 'Total School Budget', 'Per Student Budget'], axis=1)
slim_summary
school_scores = slim_summary.groupby("School Type", group_keys=False)
scores_by_school_type = school_scores.mean()
scores_by_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
